$s$

In [1]:
import ipywidgets as widgets
out = widgets.Output()
with out:
    
    %bookmark 'model_dir'
    current_dir = %pwd
    while 'models' in current_dir:
        %cd ..
        current_dir = %pwd
    %bookmark 'base_dir'
    %cd source_code
    out.clear_output()
%run base
%run gui_v6
with out:
    %cd ..
    %cd models/slider_cranck
    out.clear_output()


In [2]:
m=model()
m.show()

## VEHICLE DYNAMICS MODELING AND SIMULATION TOOL

In [22]:
from solvers import dds
q0=pd.concat([i.dic    for i in m.bodies])
qd0=pd.concat([i.qd0()    for i in m.bodies])
topology_writer(m.bodies,m.joints,[],[],'sd')
sd=dds(q0,qd0,m.bodies,m.joints,[],[],'sd',0.8,0.005)

DOF : 1 

Independent Coordinates are: ['rbs_slider.x'] with indices: [21] 

time_step: 0
[ 210.    0.]
time_step: 1
[ 210.00182442    0.72976658]


C:\Users\khale\Anaconda3\lib\site-packages\scipy\integrate\_ode.py:1035: UserWarning:

dop853: step size becomes too small



time_step: 2
[ 210.00729768    1.45953699]
time_step: 3
[ 210.01641982    2.18931893]
time_step: 4
[ 210.02919091    2.91912006]
time_step: 5
[ 210.04561108    3.64894805]
time_step: 6
[ 210.06568048    4.37881058]
time_step: 7
[ 210.08939929    5.10871528]
time_step: 8
[ 210.11676776    5.8386698 ]
time_step: 9
[ 210.14778614    6.56868176]
time_step: 10
[ 210.18245474    7.29875875]
time_step: 11
[ 210.2207739     8.02890836]
time_step: 12
[ 210.26274402    8.75913813]
time_step: 13
[ 210.30836551    9.48945559]
time_step: 14
[ 210.35763881   10.21986823]
time_step: 15
[ 210.41056444   10.95038351]
time_step: 16
[ 210.46714293   11.68100885]
time_step: 17
[ 210.52737483   12.41175163]
time_step: 18
[ 210.59126075   13.14261919]
time_step: 19
[ 210.65880135   13.87361882]
time_step: 20
[ 210.72999729   14.60475777]
time_step: 21
[ 210.80484929   15.33604323]
time_step: 22
[ 210.88335811   16.06748235]
time_step: 23
[ 210.96552452   16.79908219]
time_step: 24
[ 211.05134935   17.530849

In [3]:
r=model()
r.show()

## VEHICLE DYNAMICS MODELING AND SIMULATION TOOL

In [6]:
list(pd.concat([i.dic    for i in r.bodies])

SyntaxError: unexpected EOF while parsing (<ipython-input-6-853f7783fcd4>, line 1)

In [15]:
t=np.arange(0,2*np.pi,0.005)
act=absolute_locating('act',r.bodies['rbs_link'],'z')
act.pos_f=lambda t: -1000*np.sin(t)
actd=pd.Series([act],index=['act'])
topology_writer(r.bodies,r.joints,actd,[],'ffr7k')
rkds=kds(r.bodies,r.joints,actd,'ffr7k',t)


Running System Kinematic Analysis:
Progress: [                         ] 0% of 1257 steps.

C:\Users\khaled.ghobashy\Desktop\asurt_cdt\source_code\mbs_objects\constraints.py:697: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\khaled.ghobashy\Desktop\asurt_cdt\source_code\mbs_objects\constraints.py:698: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\khaled.ghobashy\Desktop\asurt_cdt\source_code\mbs_objects\constraints.py:720: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\khaled.ghobashy\Desktop\asurt_cdt\source_code\mbs_objects\constraints.py:721: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Users\khaled.ghobashy\Desktop\asurt_cdt\source_code\mbs_objects\constraints.py:747: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use 

Recalculating Jacobian
Recalculating Jacobian
Recalculating Jacobian
Recalculating Jacobian
Recalculating Jacobian
Recalculating Jacobian
Recalculating Jacobian
Recalculating Jacobian
Recalculating Jacobian
Recalculating Jacobian
Recalculating Jacobian
Progress: [                         ] 0% of 1257 steps.Recalculating Jacobian
Recalculating Jacobian
Recalculating Jacobian
Recalculating Jacobian
Recalculating Jacobian
Recalculating Jacobian
Recalculating Jacobian
Progress: [============             ] 49% of 1257 steps.Recalculating Jacobian
Progress: [============             ] 49% of 1257 steps.Recalculating Jacobian
Progress: [============             ] 50% of 1257 steps.Recalculating Jacobian
Progress: [============             ] 50% of 1257 steps.Recalculating Jacobian
Progress: [============             ] 50% of 1257 steps.Recalculating Jacobian
Progress: [======================== ] 99% of 1257 steps.Recalculating Jacobian
Progress: [=========================] 100% of 1257 steps.

In [16]:
xaxis  = dict(nticks=10,mirror='all',showline=True,linewidth=2,linecolor='#cdc0b0')
yaxis  = dict(nticks=10,mirror='all',showline=True,linewidth=2,linecolor='#cdc0b0')
margin = dict(l=50,r=50,b=50,t=50,pad=4)

layout = go.Layout(xaxis=xaxis,yaxis=yaxis,margin=margin,width=700,height=300,paper_bgcolor='#f8f8ff',plot_bgcolor='#f5f5f5')
data1 = go.Scatter(x=rkds[0]['rbs_link.x'],y=np.sqrt(rkds[0]['rbs_link.z']**2+rkds[0]['rbs_link.x']**2))
data2 = go.Scatter(x=t,y=rkds[1]['rbs_link.x'][0:2000],name='x')
data3 = go.Scatter(x=t,y=rkds[1]['rbs_link.z'][0:2000],name='z')

fig = go.Figure(data=[data2,data3], layout=layout)
plotly.offline.iplot(fig)

In [5]:
a,b,c,d=np.array([1,2,3,4]).reshape((4,1))

In [8]:
np.array([[a],[b],[c]]).flatten()

array([1, 2, 3])

In [16]:
a=np.array([[1,2,3],
            [4,5,6],
            [7,8,9]])
a[1,0]

4

In [8]:
from solvers import dds
q0=pd.concat([i.dic    for i in r.bodies])
qd0=pd.concat([i.qd0()    for i in r.bodies])
topology_writer(r.bodies,r.joints,[],[],'ffr4')
rds=dds(q0,qd0,r.bodies,r.joints,[],[],'ffr4',0.5,5e-4)

DOF : 1 

Independent Coordinates are: ['rbs_link.z'] with indices: [9] 

Progress: [                         ] 0% of 1000 steps.

C:\Users\khale\Anaconda3\lib\site-packages\scipy\integrate\_ode.py:1035: UserWarning:

dop853: step size becomes too small



Progress: [=========================] 100% of 1000 steps.

In [9]:
t=np.arange(0,0.8,5e-4)

xaxis  = dict(nticks=10,mirror='all',showline=True,linewidth=2,linecolor='#cdc0b0')
yaxis  = dict(nticks=10,mirror='all',showline=True,linewidth=2,linecolor='#cdc0b0')
margin = dict(l=50,r=50,b=50,t=50,pad=4)

layout = go.Layout(xaxis=xaxis,yaxis=yaxis,margin=margin,width=700,height=300,paper_bgcolor='#f8f8ff',plot_bgcolor='#f5f5f5')
data1 = go.Scatter(x=rds[0]['rbs_link.x'],y=np.sqrt(rds[0]['rbs_link.z']**2+rds[0]['rbs_link.x']**2))
data2 = go.Scatter(x=t,y=rds[2]['rbs_link.x'])
data3 = go.Scatter(x=t,y=rds[2]['rbs_link.z'])

fig = go.Figure(data=[data2,data3], layout=layout)
plotly.offline.iplot(fig)